# Applied Data Science Capstone
----------
### Week 3 Peer Graded Assignment

* ## Assignment 1
Scrape wikipedia for the table on neighborhoods in Canada and format it into a pandas DataFrame

In [29]:
#importing needed libraries
import requests
import pandas as pd
from pandas.io.json import json_normalize
#!pip install lxml              #uncomment to install lxml parser
#!pip install beautifulsoup4    #uncomment to install bs
from bs4 import BeautifulSoup

### Making url request and grabbing requested table

In [170]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
s = requests.Session()
response = s.get(url, timeout = 10)
soup = BeautifulSoup(response.content, "html.parser")

# to fix the content idents etc and make it readable
pretty_soup = soup.prettify()
# finding the correct table and its headers
table = soup.find('table', {'class' : 'wikitable sortable'})

### Extracting column names and rows from table

In [171]:
rows = table.find_all("tr")
headers = [th.text.rstrip() for th in rows[0].find_all("th")]
codes_in_example = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

# renaming Postal Code header to PostalCode, as per course instructions
headers[0] = 'PostalCode'
print(" Table Headers : ", headers)

# grabbing rows
table_rows = []
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all("td")]
    if data[1].upper()=='NOT ASSIGNED':                # Reject rows that have no Borough
        continue
    if data[2].upper()=='NOT ASSIGNED':                # If no neighborhood data exists, copy from Borough data
        data[2] = data[1]
    table_rows.append(data)

 Table Headers :  ['PostalCode', 'Borough', 'Neighbourhood']


### Creating a Pandas DataFrame with the extracted data

In [172]:
df = pd.DataFrame(data = table_rows, columns = headers)
#df.loc[df['PostalCode'] == "M5A"]     #Checking if my data matches data from example in course
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Print Specific dataframe from example

In [177]:
df.loc[df['PostalCode'].isin(codes_in_example)]               #Printing specific dataframe from example

,PostalCode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


In [120]:
df.shape

(103, 3)

***Assgnment 1 ENDS HERE***


---------------------------------


* ## Assignment 2
Get the latitude and the longitude coordinates of each neighborhood using Geocoder and add to the DataFrame for later use with Foursquare

In [131]:
#!pip install geocoder      #uncomment to install Geocoder package
import geocoder

### Fetch location data for each postal code in Toronto, Ontario from geocodefarm

In [160]:
latitudes = []
longitudes = []
for postal in df['PostalCode']:
    g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(postal))
    print(postal, ", found coordinates : ", g.latlng)
    while g.latlng == None:
        g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(postal))
        print(postal, ", found coordinates : ", g.latlng)
    latlng = g.latlng
    latitudes.append(latlng[0])
    longitudes.append(latlng[1])
print("====================")
print("======Complete======")
print("====================")

M3A , found coordinates :  [43.7561225891716, -79.3296356201129]
M4A , found coordinates :  [43.7267799377716, -79.3107376098129]
M5A , found coordinates :  [43.6553535461715, -79.365043640113]
M6A , found coordinates :  [43.7219963073716, -79.445915222113]
M7A , found coordinates :  [43.6639099121715, -79.388732910113]
M9A , found coordinates :  [43.6645202636715, -79.529075622513]
M1B , found coordinates :  [43.8070793151717, -79.2015533447129]
M3B , found coordinates :  [43.7463111877716, -79.357604980413]
M4B , found coordinates :  [43.7066764831716, -79.3065795898129]
M5B , found coordinates :  [43.6574783327152, -79.378631591713]
M6B , found coordinates :  [43.7070999145716, -79.447883605913]
M9B , found coordinates :  [43.6527595520715, -79.5523300171303]
M1C , found coordinates :  [43.7872161865717, -79.1525192260129]
M3C , found coordinates :  [43.7263832092716, -79.3407669067129]
M4C , found coordinates :  [43.6897392272715, -79.3100509643129]
M5C , found coordinates :  [43.6

### Combine location data with dataframe

In [178]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.756123,-79.329636
1,M4A,North York,Victoria Village,43.726780,-79.310738
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655354,-79.365044
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.721996,-79.445915
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663910,-79.388733


In [179]:
df.loc[df['PostalCode']=='M5A']        #Checking if my location data is similar to the one displayed on course example

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655354,-79.365044


### Print Specific dataframe from example with location data

In [180]:
df.loc[df['PostalCode'].isin(codes_in_example)]       

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655354,-79.365044
6,M1B,Scarborough,"Malvern, Rouge",43.807079,-79.201553
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706676,-79.306580
23,M4G,East York,Leaside,43.708229,-79.367691
24,M5G,Downtown Toronto,Central Bay Street,43.656761,-79.386490
27,M2H,North York,Hillcrest Village,43.802097,-79.359268
32,M1J,Scarborough,Scarborough Village,43.744732,-79.234970
50,M9L,North York,Humber Summit,43.758663,-79.560555
54,M4M,East Toronto,Studio District,43.661816,-79.342186
71,M1R,Scarborough,"Wexford, Maryvale",43.749367,-79.302498


***Assgnment 2 ENDS HERE***


---------------------------------


* ## Assignment 3
Explore and cluster the neighborhoods in Toronto. You decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. generate maps to visualize neighborhoods and clusters.

In [280]:
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
import numpy as np

In [262]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
mymap = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

### A map of all the neighborhoods in toronto (not clustered yet)

In [263]:
mask = df['Borough'].str.contains(r'Toronto', na=True)              #keeping only entries of the table that contain "Toronto" in borough section as instructed
df_tor = df[mask].reset_index()

for lat, lon, lbl in zip(df_tor.Latitude, df_tor.Longitude, df_tor.Neighbourhood):
    label = folium.Popup(lbl, parse_html=True)
    folium.CircleMarker([lat,lon], radius = 7, color = 'red', popup = label, fill = True, fill_color = 'red', fill_opacity = 0.5).add_to(mymap)
mymap

In [256]:
CLIENT_ID = '4H221GSI4E14FOF4WKROCNN3CRVOAYJZIJ2ZJQ3Q5WRBBEJZ' # your Foursquare ID
CLIENT_SECRET = 'RNRQT0MROZPVEMNOCC5HKHKXGBRWOFUH3OJJMEGLCMDWS3YD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Get venue data for each neighborhood to use for clustering

In [258]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [259]:
toronto_nearby_venues = getNearbyVenues(df_tor['Neighbourhood'], df_tor['Latitude'], df_tor['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [260]:
toronto_nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.655354,-79.365044,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
1,"Regent Park, Harbourfront",43.655354,-79.365044,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.655354,-79.365044,Tandem Coffee,43.653559,-79.361809,Coffee Shop
3,"Regent Park, Harbourfront",43.655354,-79.365044,Rooster Coffee,43.651900,-79.365609,Coffee Shop
4,"Regent Park, Harbourfront",43.655354,-79.365044,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
...,...,...,...,...,...,...,...
1994,"Business reply mail Processing Centre, South C...",43.651894,-79.381714,Second Cup,43.656027,-79.380575,Coffee Shop
1995,"Business reply mail Processing Centre, South C...",43.651894,-79.381714,Earls Kitchen & Bar,43.647946,-79.383706,Bar
1996,"Business reply mail Processing Centre, South C...",43.651894,-79.381714,Freshii,43.649273,-79.383748,Salad Place
1997,"Business reply mail Processing Centre, South C...",43.651894,-79.381714,Disney Store,43.654248,-79.381232,Toy / Game Store


In [261]:
print('There are {} unique categories.'.format(len(toronto_nearby_venues['Venue Category'].unique())))

There are 241 unique categories.


### Encoding the venue category data for use with kmeans clustering

In [337]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
tor_onehot['Neighborhood']
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.010204,0.000000,0.0000,0.010204,0.010204,0.020408,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.017857,0.0000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.0000,0.020000,0.000000,0.010000,0.010000,0.000000,0.010000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.013699,0.000000,0.0000,0.000000,0.000000,0.013699,0.000000,0.013699,0.013699,...,0.000000,0.013699,0.000000,0.013699,0.013699,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.00
5,Christie,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.012500,0.000000,0.0125,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000,0.000000,0.00
7,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.0000,0.030000,0.000000,0.010000,0.000000,0.000000,0.010000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01
8,Davisville,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.014925,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.014925,0.000000,0.014925,0.014925,0.014925,0.000000,0.00


### Make table with top 10 venues for each neighborhood

In [338]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [339]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Italian Restaurant,Japanese Restaurant,Café,Seafood Restaurant,Beer Bar,Pub,Restaurant,Hotel,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Café,Restaurant,Gift Shop,Coffee Shop,Bakery,Arts & Crafts Store,Thrift / Vintage Store,Art Gallery,Lounge
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Café,Clothing Store,Gym,Gastropub,Restaurant,Theater,Plaza,Bar
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Italian Restaurant,Restaurant,Hotel,Grocery Store,Park,Bar,Gym,Beer Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Hotel,Sushi Restaurant,Sandwich Place,Japanese Restaurant,Juice Bar,Bookstore


In [340]:
# set number of clusters
kclusters = 5
tor_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [341]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_tor

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
tor_merged.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
tor_merged.drop("index",1,inplace=True)
tor_merged.drop("PostalCode",1 ,inplace=True)
tor_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.655354,-79.365044,0,Coffee Shop,Breakfast Spot,Italian Restaurant,Yoga Studio,Bakery,Dance Studio,Diner,Discount Store,Electronics Store,Sandwich Place
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663910,-79.388733,0,Coffee Shop,Gym,Yoga Studio,Bubble Tea Shop,Café,Martial Arts School,Ethiopian Restaurant,Mexican Restaurant,Escape Room,Chinese Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657478,-79.378632,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Furniture / Home Store,Theater,Burger Joint,Ramen Restaurant,Bookstore
3,Downtown Toronto,St. James Town,43.651112,-79.375732,0,Coffee Shop,Seafood Restaurant,Café,Cocktail Bar,Restaurant,American Restaurant,Beer Bar,Gastropub,Lingerie Store,Japanese Restaurant
4,East Toronto,The Beaches,43.676559,-79.294159,4,Pub,Trail,Health Food Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Women's Store


### Create folium map with clusters

In [343]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

***Assgnment 3 ENDS HERE***


---------------------------------